In [69]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#%load_ext autotime
import geopandas as gpd
import geopy
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook
import reverse_geocoder as rg
import os

In [70]:
#usw1= pd.read_csv('../data/7890488/USC00042863.csv')

file_name = os.path.basename('USC00042863.csv')

usw1['station'] = file_name

usw1 = pd.DataFrame(usw1)

usw1

,Unnamed: 0,Date,tmax,tmin,prcp,station
0,1,1894-01-01,60.0,41.0,0.00,USC00042863.csv
1,2,1894-01-02,58.0,50.0,0.40,USC00042863.csv
2,3,1894-01-03,57.0,42.0,0.00,USC00042863.csv
3,4,1894-01-04,53.0,42.0,0.28,USC00042863.csv
4,5,1894-01-05,50.0,38.0,0.00,USC00042863.csv
...,...,...,...,...,...,...
46746,46747,2021-12-27,NaN,NaN,NaN,USC00042863.csv
46747,46748,2021-12-28,NaN,NaN,NaN,USC00042863.csv
46748,46749,2021-12-29,NaN,NaN,NaN,USC00042863.csv
46749,46750,2021-12-30,NaN,NaN,NaN,USC00042863.csv


In [71]:
# List all the CSV files in the directory
csv_files = [file for file in os.listdir('../data/7890488') if file.endswith('.csv')]

dfs = []

# Iterate over CSV files
for file in csv_files:
   
    df = pd.read_csv('../data/7890488/' + file)
    file_name = os.path.basename(file)
    
    df['station'] = file_name
    dfs.append(df)
uswd = pd.concat(dfs, ignore_index=True)

uswd

,Unnamed: 0,Date,tmax,tmin,prcp,station
0,1,1894-01-01,60.0,41.0,0.00,USC00042863.csv
1,2,1894-01-02,58.0,50.0,0.40,USC00042863.csv
2,3,1894-01-03,57.0,42.0,0.00,USC00042863.csv
3,4,1894-01-04,53.0,42.0,0.28,USC00042863.csv
4,5,1894-01-05,50.0,38.0,0.00,USC00042863.csv
...,...,...,...,...,...,...
10475909,46747,2021-12-27,32.0,1.2,0.22,USW00094973.csv
10475910,46748,2021-12-28,22.1,-5.8,0.06,USW00094973.csv
10475911,46749,2021-12-29,20.1,-4.9,0.00,USW00094973.csv
10475912,46750,2021-12-30,NaN,NaN,NaN,USW00094973.csv


In [72]:
#clean up file name column
uswd['station'] = uswd['station'].str.rstrip('.csv')
uswd

,Unnamed: 0,Date,tmax,tmin,prcp,station
0,1,1894-01-01,60.0,41.0,0.00,USC00042863
1,2,1894-01-02,58.0,50.0,0.40,USC00042863
2,3,1894-01-03,57.0,42.0,0.00,USC00042863
3,4,1894-01-04,53.0,42.0,0.28,USC00042863
4,5,1894-01-05,50.0,38.0,0.00,USC00042863
...,...,...,...,...,...,...
10475909,46747,2021-12-27,32.0,1.2,0.22,USW00094973
10475910,46748,2021-12-28,22.1,-5.8,0.06,USW00094973
10475911,46749,2021-12-29,20.1,-4.9,0.00,USW00094973
10475912,46750,2021-12-30,NaN,NaN,NaN,USW00094973


In [73]:
uswd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10475914 entries, 0 to 10475913
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   Date        object 
 2   tmax        float64
 3   tmin        float64
 4   prcp        float64
 5   station     object 
dtypes: float64(3), int64(1), object(2)
memory usage: 479.5+ MB


In [122]:
#formatting city data to merge into weather
city = pd.read_csv('../data/city_info.csv')
city.drop(['Unnamed: 0', 'Name', 'Stn.Name', 'Stn.stDate', 'Stn.edDate'],axis=1, inplace=True)
city.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      461 non-null    object 
 1   Lat     461 non-null    float64
 2   Lon     461 non-null    float64
dtypes: float64(2), object(1)
memory usage: 10.9+ KB


In [75]:
lat = city.Lat
lon = city.Lon

result1 = []
for i in range(len(lat)):
     #Adding to result based on indexes
     result1.append((lat[i], lon[i]))

In [76]:
location_dic1 = rg.search(result1)

In [77]:
location_df1 = pd.DataFrame(location_dic1)
location_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lat     461 non-null    object
 1   lon     461 non-null    object
 2   name    461 non-null    object
 3   admin1  461 non-null    object
 4   admin2  461 non-null    object
 5   cc      461 non-null    object
dtypes: object(6)
memory usage: 21.7+ KB


In [78]:
location_df1.head()

,lat,lon,name,admin1,admin2,cc
0,42.83301,-108.73067,Lander,Wyoming,Fremont County,US
1,42.83301,-108.73067,Lander,Wyoming,Fremont County,US
2,41.13998,-104.82025,Cheyenne,Wyoming,Laramie County,US
3,41.13998,-104.82025,Cheyenne,Wyoming,Laramie County,US
4,44.90969,-89.60457,Schofield,Wisconsin,Marathon County,US


In [127]:
location_df1['lat'] = location_df1['lat'].astype(float)
location_df1['lon'] = location_df1['lon'].astype(float)
city['lat'] = city['Lat']
city['lon'] = city['Lon']

In [139]:
location_df1['lat']= location_df1['lat'].round(decimals=6)
location_df1['lon']= location_df1['lon'].round(decimals=6)
city = city.round(decimals =6)

In [129]:
city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      461 non-null    object 
 1   Lat     461 non-null    float64
 2   Lon     461 non-null    float64
 3   lat     461 non-null    float64
 4   lon     461 non-null    float64
dtypes: float64(4), object(1)
memory usage: 18.1+ KB


In [109]:
location_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     461 non-null    float64
 1   lon     461 non-null    float64
 2   name    461 non-null    object 
 3   admin1  461 non-null    object 
 4   admin2  461 non-null    object 
 5   cc      461 non-null    object 
dtypes: float64(2), object(4)
memory usage: 21.7+ KB


In [141]:
station = city.merge(location_df1, how = 'inner', on = ['lat', 'lon'])
station

,ID,Lat,Lon,lat,lon,name,admin1,admin2,cc
0,USW00012924,27.80,-97.40,27.80,-97.40,Corpus Christi,Texas,Nueces County,US
1,USW00012924,27.80,-97.40,27.80,-97.40,Corpus Christi,Texas,Nueces County,US
2,USW00012924,27.80,-97.40,27.80,-97.40,Corpus Christi,Texas,Nueces County,US
3,USW00012924,27.80,-97.40,27.80,-97.40,Corpus Christi,Texas,Nueces County,US
4,USW00094728,40.78,-73.97,40.78,-73.97,Manhattan,New York,New York County,US
5,USC00280734,40.83,-75.08,40.83,-75.08,Belvidere,New Jersey,Warren County,US
6,USC00280734,40.83,-75.08,40.83,-75.08,Belvidere,New Jersey,Warren County,US
7,USC00280734,40.83,-75.08,40.83,-75.08,Belvidere,New Jersey,Warren County,US
8,USC00280734,40.83,-75.08,40.83,-75.08,Belvidere,New Jersey,Warren County,US
9,USC00042863,33.12,-117.09,33.12,-117.09,Escondido,California,San Diego County,US
